In [ ]:
import module_loader
import pandas as pd
from modules.curves import *
from modules.bsplines import BSpline
from modules.dual import Dual
import matplotlib.pyplot as plt

### Example 1

The demonstration of the mixed interpolator.

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,
    datetime(2023, 1, 1): 1.00,
    datetime(2024, 1, 1): 1.00,
    datetime(2025, 1, 1): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 3, 3): 1.0,
    Swap(datetime(2022, 1, 1), 12*2, 3, 3): 1.5,
    Swap(datetime(2022, 1, 1), 12*3, 3, 3): 2.0,
}
t = [
    datetime(2023, 1, 1), datetime(2023, 1, 1), datetime(2023, 1, 1), datetime(2023, 1, 1),
    datetime(2024, 1, 1),
    datetime(2025, 1, 1), datetime(2025, 1, 1), datetime(2025, 1, 1), datetime(2025, 1, 1),
]
s_cv = SolvedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    algorithm="levenberg_marquardt"
)
adv_cv = AdvancedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    t=t, algorithm="levenberg_marquardt"
)
print(s_cv.iterate())
print(adv_cv.iterate())

In [ ]:
fig, ax = plt.subplots()

x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*3)]
ax.set_ylim(0.9, 5)

y = [adv_cv.rate(date, days=1).real for date in x]
z = [s_cv.rate(date, days=1).real for date in x]
ax.plot(x, z)
ax.plot(x, y)
plt.show()

### Example 2

A layered curveset with inherited rates yields the same curve.

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,
    datetime(2023, 1, 1): 1.00,
    datetime(2024, 1, 1): 1.00,
    datetime(2025, 1, 1): 1.00,
    datetime(2027, 1, 1): 1.00,
    datetime(2029, 1, 1): 1.00,
    datetime(2032, 1, 1): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 3, 3): 1.0,
    Swap(datetime(2022, 1, 1), 12*2, 3, 3): 1.4,
    Swap(datetime(2022, 1, 1), 12*3, 3, 3): 1.64,
    Swap(datetime(2022, 1, 1), 12*5, 3, 3): 1.84,
    Swap(datetime(2022, 1, 1), 12*7, 3, 3): 1.90,
    Swap(datetime(2022, 1, 1), 12*10, 3, 3): 1.97,
}
t = [
    datetime(2022, 1, 1), datetime(2022, 1, 1), datetime(2022, 1, 1), datetime(2022, 1, 1),
    datetime(2023, 1, 1),
    datetime(2024, 1, 1),
    datetime(2025, 1, 1),
    datetime(2027, 1, 1),
    datetime(2029, 1, 1),
    datetime(2032, 1, 1), datetime(2032, 1, 1), datetime(2032, 1, 1), datetime(2032, 1, 1),
]
adv_cv = AdvancedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    t=t, algorithm="levenberg_marquardt"
)
print(adv_cv.iterate())

In [ ]:
fig, ax = plt.subplots()
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*10)]
y = [adv_cv.rate(date, days=1).real for date in x]
ax.plot(x, y)
plt.show()

Now add some additional instruments and skew them derived from the first layer

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,
    datetime(2023, 1, 1): 1.00,
    datetime(2024, 1, 1): 1.00,
    datetime(2025, 1, 1): 1.00,
    datetime(2026, 1, 1): 1.00,
    datetime(2027, 1, 1): 1.00,
    datetime(2028, 1, 1): 1.00,
    datetime(2029, 1, 1): 1.00,
    datetime(2030, 1, 1): 1.00,
    datetime(2031, 1, 1): 1.00,
    datetime(2032, 1, 1): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*2, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*3, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*4, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*5, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*6, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*7, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*8, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*9, 3, 3): 0,
    Swap(datetime(2022, 1, 1), 12*10, 3, 3): 0,
}
skews = {4: 0.001, 6: 0.001, 8: 0.002, 9:0.0025}
for i, swap in enumerate(swaps.keys()):
    bump = 0 if (i+1) not in skews.keys() else skews[i+1]
    swaps[swap] = swap.rate(adv_cv).real + bump

t = [
    datetime(2022, 1, 1), datetime(2022, 1, 1), datetime(2022, 1, 1), datetime(2022, 1, 1),
    datetime(2023, 1, 1),
    datetime(2024, 1, 1),
    datetime(2025, 1, 1),
    datetime(2026, 1, 1),
    datetime(2027, 1, 1),
    datetime(2028, 1, 1),
    datetime(2029, 1, 1),
    datetime(2030, 1, 1),
    datetime(2031, 1, 1),
    datetime(2032, 1, 1), datetime(2032, 1, 1), datetime(2032, 1, 1), datetime(2032, 1, 1),
]
adv_cv2 = AdvancedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    t=t, algorithm="levenberg_marquardt"
)
print(adv_cv2.iterate())

In [ ]:
fig, ax = plt.subplots()
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*10)]
ax.set_ylim(1.8, 2.5)
y = [adv_cv.rate(date, days=1).real for date in x]
z = [adv_cv2.rate(date, days=1).real for date in x]
ax.plot(x, np.array(z))
ax.plot(x, np.array(y))
plt.show()

In [ ]:
df_first_layer_risk = pd.DataFrame([], index=["1y", "2y", "3y", "5y", "7y", "10y"], columns=[1,2,3,4,5,6,7,8,9,10])
for i, swap in enumerate(swaps):
    df_first_layer_risk[i+1] = swap.risk(adv_cv)
df_first_layer_risk.style.format("{:,.2f}")

In [ ]:
df_second_layer_risk = pd.DataFrame([], index=["1y", "2y", "3y", "4y", "5y", "6y", "7y", "8y", "9y", "10y"], columns=[1,2,3,4,5,6,7,8,9,10])
for i, swap in enumerate(swaps):
    df_second_layer_risk[i+1] = swap.risk(adv_cv2)
df_second_layer_risk.style.format("{:,.2f}")

### Example 3

Here, after construction we insert a turn which is a either one new swap and one new node (if the turn is exactly before or after an existing node) or two new swaps and two new nodes. *Turns implemented in this way do not work with log-cubic interpolation*.

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,
    datetime(2022, 12, 31): 1.00,
    datetime(2023, 1, 1): 1.00,
    datetime(2023, 6, 30): 1.00,
    datetime(2023, 7, 1): 1.00,
    datetime(2024, 1, 1): 1.00,
    datetime(2025, 1, 1): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 3, 3): 1.0,
    Swap(datetime(2022, 1, 1), 12*2, 3, 3): 1.5,
    Swap(datetime(2022, 1, 1), 12*3, 3, 3): 2.0,
    SwapSpread(
        Swap(datetime(2022, 12, 30), 1, 1, 1, days=True),
        Swap(datetime(2022, 12, 31), 1, 1, 1, days=True)
    ): -0.25,
    SwapSpread(
        Swap(datetime(2023, 6, 29), 1, 1, 1, days=True),
        Swap(datetime(2023, 6, 30), 1, 1, 1, days=True)
    ): -0.25,
    SwapSpread(
        Swap(datetime(2023, 6, 30), 1, 1, 1, days=True),
        Swap(datetime(2023, 7, 1), 1, 1, 1, days=True)
    ): 0.25,
}
s_cv = SolvedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

In [ ]:
fig, ax = plt.subplots()
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*3)]
y = [s_cv.rate(date, days=1).real for date in x]
z = [s_cv.rate(date, days=91).real for date in x]
ax.plot(x, y)
# ax.plot(x, z)
# ax.set_xlim(datetime(2023,1,1),datetime(2023,9,30))
# ax.set_ylim(1.95,2.05)
plt.xticks(rotation=90)
plt.show()

### Example 5

We demonstrate the effects of keeping constant knot points and node points for the cubic spline as opposed to daily variation aligning with par tenor rates.

First the static curve constructed with IMM nodes and IMM knots.

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,
    datetime(2023, 3, 15): 1.00,
    datetime(2024, 3, 15): 1.00,
    datetime(2025, 3, 15): 1.00,
    datetime(2026, 3, 15): 1.00,
    datetime(2027, 3, 15): 1.00,
    datetime(2028, 3, 15): 1.00,
    datetime(2029, 3, 15): 1.00,
    datetime(2030, 3, 15): 1.00,
    datetime(2031, 3, 15): 1.00,
    datetime(2032, 3, 15): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 3, 3): 1.00,
    Swap(datetime(2022, 1, 1), 12*2, 3, 3): 1.04,
    Swap(datetime(2022, 1, 1), 12*3, 3, 3): 1.09,
    Swap(datetime(2022, 1, 1), 12*4, 3, 3): 1.12,
    Swap(datetime(2022, 1, 1), 12*5, 3, 3): 1.127,
    Swap(datetime(2022, 1, 1), 12*6, 3, 3): 1.13,
    Swap(datetime(2022, 1, 1), 12*7, 3, 3): 1.132,
    Swap(datetime(2022, 1, 1), 12*8, 3, 3): 1.13,
    Swap(datetime(2022, 1, 1), 12*9, 3, 3): 1.127,
    Swap(datetime(2022, 1, 1), 12*10, 3, 3): 1.123,
}

t = [
    datetime(2023, 3, 15), datetime(2023, 3, 15), datetime(2023, 3, 15), datetime(2023, 3, 15),
    datetime(2024, 3, 15), 
    datetime(2025, 3, 15),
    datetime(2026, 3, 15),
    datetime(2027, 3, 15),
    datetime(2028, 3, 15),
    datetime(2029, 3, 15),
    datetime(2030, 3, 15),
    datetime(2031, 3, 15),
    datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15),
]
adv_cv2 = AdvancedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    t=t, algorithm="levenberg_marquardt"
)
print(adv_cv2.iterate())

Now we collect swap rate data for further dates **assuming** rates evolve **exactly** as predicted. These will be used to perform the experiment to analyse the effect of changing nodes and knots. We acquire swap data for the 1st Jan 2022, and the subsequent 70 days.

In [ ]:
df = pd.DataFrame([], index=[datetime(2022, 1, 1)+ timedelta(days=i) for i in range(73)],
                  columns=[12*1, 12*2, 12*3, 12*4, 12*5, 12*6, 12*7, 12*8, 12*9, 12*10])
for date in df.index:
    for tenor in df.columns:
        df.loc[date, tenor] = Swap(date, tenor, 3, 3).rate(adv_cv2).real
        
df

Now we will build curves for each of the 70 days using the forecast rates predicted by the above curve.
 
  - Nodes will align with the per tenor end rates.
  - Knots for the log-cubic spline begin in 1 year and extend to the 10 year tenor.

In [ ]:
def add_years(date: datetime, years: int) -> datetime:
    return date.replace(day=date.day, month=date.month, year=date.year+years)

test_swaps = [
    Swap(datetime(2026,4,15), 6, 6, 6),
    Swap(datetime(2027,8,15), 6, 6, 6),
    Swap(datetime(2028,2,15), 6, 6, 6),
    Swap(datetime(2029,6,15), 6, 6, 6),
    Swap(datetime(2030,7,15), 6, 6, 6),
]
test_df = pd.DataFrame([], index=df.index, columns=[0,1,2,3,4]) 

for date in df.index:
    nodes = {add_years(date, i): 1.00 for i in range(len(df.columns)+1)}
    nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}
    swaps = {Swap(date, tenor, 3, 3): df.loc[date, tenor] for tenor in df.columns}
    t = [add_years(date, 1), add_years(date, 1), add_years(date, 1)]
    t += [add_years(date, i+1) for i in range(len(df.columns))]
    t += [add_years(date, 10), add_years(date, 10), add_years(date, 10)]
    adv_cv2 = AdvancedCurve(
        nodes=nodes_dual, interpolation="log_linear", 
        swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
        t=t, algorithm="gauss_newton"
    )
    print(date, adv_cv2.iterate())
    for col, swap in enumerate(test_swaps):
        test_df.loc[date, col] = swap.rate(adv_cv2).real

test_df

In [ ]:
initial = test_df.iloc[0, :].copy()
for col in test_df:
    test_df.loc[:, col] -= test_df.loc[test_df.index[0], col]
test_df.plot()

The following is included as a numerical demonstration of the mathematical fact that using consistent knots and consistent data for differenet datasites the spline is identical and the test swaps, as with all other swaps maintain their values throughout the period.

In [ ]:
nodes = {
    datetime(2023, 3, 15): 1.00,
    datetime(2024, 3, 15): 1.00,
    datetime(2025, 3, 15): 1.00,
    datetime(2026, 3, 15): 1.00,
    datetime(2027, 3, 15): 1.00,
    datetime(2028, 3, 15): 1.00,
    datetime(2029, 3, 15): 1.00,
    datetime(2030, 3, 15): 1.00,
    datetime(2031, 3, 15): 1.00,
    datetime(2032, 3, 15): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i+1}": 1}) for i, (k,v) in enumerate(nodes.items())}
t = [
    datetime(2023, 3, 15), datetime(2023, 3, 15), datetime(2023, 3, 15), datetime(2023, 3, 15),
    datetime(2024, 3, 15), 
    datetime(2025, 3, 15),
    datetime(2026, 3, 15),
    datetime(2027, 3, 15),
    datetime(2028, 3, 15),
    datetime(2029, 3, 15),
    datetime(2030, 3, 15),
    datetime(2031, 3, 15),
    datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15),
]

test_df_static = pd.DataFrame([], index=df.index, columns=[0,1,2,3,4]) 

for date in df.index:
    swaps = {Swap(date, tenor, 3, 3): df.loc[date, tenor] for tenor in df.columns}
    nodes_ = {date: Dual(1.0, {"v0": 1}), **nodes_dual}
    adv_cv2 = AdvancedCurve(
        nodes=nodes_, interpolation="log_linear", 
        swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
        t=t, algorithm="gauss_newton"
    )
    print(date, adv_cv2.iterate())
    for col, swap in enumerate(test_swaps):
        test_df_static.loc[date, col] = swap.rate(adv_cv2).real

test_df_static

### Example 6

Finally let's suppose that an IMM roll has taken place, so that the knot points shift significantly but not the datasites.

In [ ]:
mar_imm_nodes = {
    datetime(2022, 3, 14): 1.00,
    datetime(2023, 3, 15): 1.00,
    datetime(2024, 3, 15): 1.00,
    datetime(2025, 3, 15): 1.00,
    datetime(2026, 3, 15): 1.00,
    datetime(2027, 3, 15): 1.00,
    datetime(2028, 3, 15): 1.00,
    datetime(2029, 3, 15): 1.00,
    datetime(2030, 3, 15): 1.00,
    datetime(2031, 3, 15): 1.00,
    datetime(2032, 3, 15): 1.00,
}
mar_imm_nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(mar_imm_nodes.items())}

jun_imm_nodes = {
    datetime(2022, 3, 14): 1.00,
    datetime(2023, 6, 15): 1.00,
    datetime(2024, 6, 15): 1.00,
    datetime(2025, 6, 15): 1.00,
    datetime(2026, 6, 15): 1.00,
    datetime(2027, 6, 15): 1.00,
    datetime(2028, 6, 15): 1.00,
    datetime(2029, 6, 15): 1.00,
    datetime(2030, 6, 15): 1.00,
    datetime(2031, 6, 15): 1.00,
    datetime(2032, 6, 15): 1.00,
}
jun_imm_nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(jun_imm_nodes.items())}

t_mar = [
    datetime(2022, 3, 14), datetime(2022, 3, 14), datetime(2022, 3, 14), datetime(2022, 3, 14),
    datetime(2023, 3, 15),
    datetime(2024, 3, 15),
    datetime(2025, 3, 15),
    datetime(2026, 3, 15),
    datetime(2027, 3, 15),
    datetime(2028, 3, 15),
    datetime(2029, 3, 15),
    datetime(2030, 3, 15),
    datetime(2031, 3, 15),
    datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15), datetime(2032, 3, 15),
]

t_jun = [
    datetime(2022, 3, 14), datetime(2022, 3, 14), datetime(2022, 3, 14), datetime(2022, 3, 14),
    datetime(2023, 6, 15),
    datetime(2024, 6, 15),
    datetime(2025, 6, 15),
    datetime(2026, 6, 15),
    datetime(2027, 6, 15),
    datetime(2028, 6, 15),
    datetime(2029, 6, 15),
    datetime(2030, 6, 15),
    datetime(2031, 6, 15),
    datetime(2032, 6, 15), datetime(2032, 6, 15), datetime(2032, 6, 15), datetime(2032, 6, 15),
]

date = datetime(2022, 3, 14)
swaps = {Swap(date, tenor, 3, 3): df.loc[date, tenor] for tenor in df.columns}
adv_cv_mar = AdvancedCurve(
        nodes=mar_imm_nodes_dual, interpolation="log_linear", 
        swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
        t=t_mar, algorithm="gauss_newton"
    )
adv_cv_jun = AdvancedCurve(
        nodes=jun_imm_nodes_dual, interpolation="log_linear", 
        swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
        t=t_jun, algorithm="gauss_newton"
    )

print(adv_cv_mar.iterate())
print(adv_cv_jun.iterate())

fig, ax = plt.subplots()
ax2 = ax.twinx()
x = [datetime(2022,4,1) + i * timedelta(days=1) for i in range(365*9)]
y = [adv_cv_mar.rate(date, days=1).real for date in x]
z = [adv_cv_jun.rate(date, days=1).real for date in x]
ax.plot(x, y)
ax.plot(x, z)
ax2.plot(x, np.array(y)-np.array(z), c=(1,0,0,0.5))
plt.xticks(rotation=90)
plt.show()

# A practical curveset

## Initial exploration

In [ ]:
nodes = {
    datetime(2022, 1, 1): 1.00,  # today's DF
    datetime(2022, 2, 3): 1.00,  # defined MPC dates..
    datetime(2022, 3, 17): 1.00,
    datetime(2022, 5, 5): 1.00,
    datetime(2022, 6, 16): 1.00,
    datetime(2022, 8, 4): 1.00,
    datetime(2022, 9, 15): 1.00,
    datetime(2022, 11, 3): 1.00,
    datetime(2022, 12, 15): 1.00,
    datetime(2023, 2, 2): 1.00,  # provisional MPC dates..
    datetime(2023, 3, 23): 1.00,
    datetime(2023, 5, 11): 1.00,
    datetime(2023, 6, 22): 1.00,
    datetime(2023, 8, 3): 1.00,
    datetime(2023, 9, 21): 1.00,
    datetime(2023, 11, 2): 1.00,
    datetime(2023, 12, 14): 1.00,
    datetime(2024, 2, 8): 1.00,  # estimated MPC dates..
    datetime(2024, 3, 21): 1.00,
    datetime(2024, 5, 16): 1.00,
    datetime(2024, 6, 20): 1.00,
    datetime(2024, 8, 8): 1.00,
    datetime(2024, 9, 19): 1.00,
    datetime(2024, 11, 7): 1.00,
    datetime(2024, 12, 12): 1.00,
    datetime(2025, 3, 19): 1.00,  # long term tenors..
    datetime(2027, 3, 17): 1.00,
    datetime(2029, 3, 15): 1.00,
    datetime(2032, 3, 15): 1.00,
    datetime(2037, 3, 15): 1.00,
    datetime(2042, 3, 15): 1.00,
    datetime(2052, 3, 15): 1.00,
    datetime(2062, 3, 15): 1.00,
    datetime(2072, 3, 15): 1.00,
}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}

ini_swaps = {
    Swap(datetime(2022, 1, 1), 34, 34, 34, days=True): 0.695,
    Swap(datetime(2022, 2, 3), 42, 42, 42, days=True): 0.95,
    Swap(datetime(2022, 3, 16), 3, 3, 3): 1.40,
    Swap(datetime(2022, 6, 15), 3, 3, 3): 1.89,
    Swap(datetime(2022, 9, 16), 3, 3, 3): 2.245,
    Swap(datetime(2022, 12, 21), 3, 3, 3): 2.53,
    Swap(datetime(2023, 3, 15), 3, 3, 3): 2.69,
    Swap(datetime(2023, 6, 21), 3, 3, 3): 2.69,
    Swap(datetime(2023, 9, 20), 3, 3, 3): 2.62,
    Swap(datetime(2023, 12, 20), 3, 3, 3): 2.5,
    Swap(datetime(2024, 3, 20), 3, 3, 3): 2.375,
    Swap(datetime(2024, 6, 19), 3, 3, 3): 2.27,
    Swap(datetime(2024, 9, 18), 3, 3, 3): 2.215,
    Swap(datetime(2024, 12, 18), 3, 3, 3): 2.17,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 2.195,
    Swap(datetime(2022, 1, 1), 12*7, 12, 12): 2.193,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 2.186,
    Swap(datetime(2022, 1, 1), 12*15, 12, 12): 2.181,
    Swap(datetime(2022, 1, 1), 12*20, 12, 12): 2.162,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 2.12,
    Swap(datetime(2022, 1, 1), 12*40, 12, 12): 2.10,
    Swap(datetime(2022, 1, 1), 12*50, 12, 12): 2.09,
}
ini_s_cv = SolvedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(ini_swaps.keys()), obj_rates=list(ini_swaps.values()),
    algorithm="levenberg_marquardt", 
)
print(ini_s_cv.iterate())

In [ ]:
fig, ax = plt.subplots()
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*10)]
z = [ini_s_cv.rate(date, days=1).real for date in x]
ax.plot(x, z)
# ax.set_xlim(datetime(2023,1,1),datetime(2024,9,30))
# ax.set_ylim(2.3,2.75)
plt.xticks(rotation=90)
plt.show()

The problem with this curve is that it is underspecified in terms of market instruments for all the numbers of nodes it contains. We seek to restrict the curvature.

## Curvature constraints in the log-linear section

In [ ]:
mpc_1 = Swap(datetime(2022, 2, 3), 1, 1, 1, days=True)
mpc_2 = Swap(datetime(2022, 3, 17), 1, 1, 1, days=True)
mpc_3 = Swap(datetime(2022, 5, 5), 1, 1, 1, days=True)
mpc_4 = Swap(datetime(2022, 6, 16), 1, 1, 1, days=True)
mpc_5 = Swap(datetime(2022, 8, 4), 1, 1, 1, days=True)
mpc_6 = Swap(datetime(2022, 9, 15), 1, 1, 1, days=True)
mpc_7 = Swap(datetime(2022, 11, 3), 1, 1, 1, days=True)
mpc_8 = Swap(datetime(2022, 12, 15), 1, 1, 1, days=True)
mpc_9 = Swap(datetime(2023, 2, 2), 1, 1, 1, days=True)
mpc_10 = Swap(datetime(2023, 3, 23), 1, 1, 1, days=True)
mpc_11 = Swap(datetime(2023, 5, 11), 1, 1, 1, days=True)
mpc_12 = Swap(datetime(2023, 6, 22), 1, 1, 1, days=True)
mpc_13 = Swap(datetime(2023, 8, 3), 1, 1, 1, days=True)
mpc_14 = Swap(datetime(2023, 9, 21), 1, 1, 1, days=True)
mpc_15 = Swap(datetime(2023, 11, 2), 1, 1, 1, days=True)
mpc_16 = Swap(datetime(2023, 12, 14), 1, 1, 1, days=True)
mpc_17 = Swap(datetime(2024, 2, 8), 1, 1, 1, days=True)
mpc_18 = Swap(datetime(2024, 3, 21), 1, 1, 1, days=True)
mpc_19 = Swap(datetime(2024, 5, 16), 1, 1, 1, days=True)
mpc_20 = Swap(datetime(2024, 6, 20), 1, 1, 1, days=True)
mpc_21 = Swap(datetime(2024, 8, 8), 1, 1, 1, days=True)
mpc_22 = Swap(datetime(2024, 9, 19), 1, 1, 1, days=True)
mpc_23 = Swap(datetime(2024, 11, 7), 1, 1, 1, days=True)
mpc_24 = Swap(datetime(2024, 12, 12), 1, 1, 1, days=True)
mpc_25 = Swap(datetime(2025, 3, 20), 1, 1, 1, days=True)

curvature_conds = {
SwapSpread(SwapSpread(mpc_1, mpc_2), SwapSpread(mpc_2, mpc_3)): 0,
SwapSpread(SwapSpread(mpc_2, mpc_3), SwapSpread(mpc_3, mpc_4)): 0,
SwapSpread(SwapSpread(mpc_3, mpc_4), SwapSpread(mpc_4, mpc_5)): 0,
SwapSpread(SwapSpread(mpc_4, mpc_5), SwapSpread(mpc_5, mpc_6)): 0,
SwapSpread(SwapSpread(mpc_5, mpc_6), SwapSpread(mpc_6, mpc_7)): 0,
SwapSpread(SwapSpread(mpc_6, mpc_7), SwapSpread(mpc_7, mpc_8)): 0,
SwapSpread(SwapSpread(mpc_7, mpc_8), SwapSpread(mpc_8, mpc_9)): 0,
SwapSpread(SwapSpread(mpc_8, mpc_9), SwapSpread(mpc_9, mpc_10)): 0,
SwapSpread(SwapSpread(mpc_9, mpc_10), SwapSpread(mpc_10, mpc_11)): 0,
SwapSpread(SwapSpread(mpc_10, mpc_11), SwapSpread(mpc_11, mpc_12)): 0,
SwapSpread(SwapSpread(mpc_11, mpc_12), SwapSpread(mpc_12, mpc_13)): 0,
SwapSpread(SwapSpread(mpc_12, mpc_13), SwapSpread(mpc_13, mpc_14)): 0,
SwapSpread(SwapSpread(mpc_13, mpc_14), SwapSpread(mpc_14, mpc_15)): 0,
SwapSpread(SwapSpread(mpc_14, mpc_15), SwapSpread(mpc_15, mpc_16)): 0,
SwapSpread(SwapSpread(mpc_15, mpc_16), SwapSpread(mpc_16, mpc_17)): 0,
SwapSpread(SwapSpread(mpc_16, mpc_17), SwapSpread(mpc_17, mpc_18)): 0,
SwapSpread(SwapSpread(mpc_17, mpc_18), SwapSpread(mpc_18, mpc_19)): 0,
SwapSpread(SwapSpread(mpc_18, mpc_19), SwapSpread(mpc_19, mpc_20)): 0,
SwapSpread(SwapSpread(mpc_19, mpc_20), SwapSpread(mpc_20, mpc_21)): 0,
SwapSpread(SwapSpread(mpc_20, mpc_21), SwapSpread(mpc_21, mpc_22)): 0,
SwapSpread(SwapSpread(mpc_21, mpc_22), SwapSpread(mpc_22, mpc_23)): 0,
SwapSpread(SwapSpread(mpc_22, mpc_23), SwapSpread(mpc_23, mpc_24)): 0,
SwapSpread(SwapSpread(mpc_23, mpc_24), SwapSpread(mpc_24, mpc_25)): 0,
}
 
swaps = {**ini_swaps, **curvature_conds}
s_cv = SolvedCurve(
    nodes=nodes_dual, interpolation="log_linear",
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    algorithm="levenberg_marquardt", w = [1]*len(ini_swaps) + [0.0001]*len(curvature_conds)
)
print(s_cv.iterate())

In [ ]:
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*50)]
z = [s_cv.rate(date, days=1).real for date in x]

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, z)
# ax.set_xlim(datetime(2023,1,1),datetime(2030,9,30))
plt.xticks(rotation=90)
plt.show()

## Turns

Next we add nodes according to the material in the text

In [ ]:
nodes_short = {
    datetime(2022, 1, 1): 1.00,  # today's DF
    datetime(2022, 2, 3): 1.00,  # defined MPC dates..
    datetime(2022, 3, 17): 1.00,
    datetime(2022, 3, 31): 1.00, # turn1
    datetime(2022, 4, 1): 1.00,  # turn1
    datetime(2022, 5, 5): 1.00,
    datetime(2022, 6, 16): 1.00,
    datetime(2022, 6, 30): 1.00, # turn2
    datetime(2022, 7, 1): 1.00,  # turn2
    datetime(2022, 8, 4): 1.00,
    datetime(2022, 9, 15): 1.00,
    datetime(2022, 9, 30): 1.00,  # turn3
    datetime(2022, 10, 1): 1.00,  # turn3
    datetime(2022, 11, 3): 1.00,
    datetime(2022, 12, 15): 1.00,
    datetime(2022, 12, 31): 1.00, # turn4
    datetime(2023, 1, 1): 1.00,  # turn4
    datetime(2023, 2, 2): 1.00,  # provisional MPC dates..
    datetime(2023, 3, 23): 1.00,
    datetime(2023, 3, 31): 1.00, # turn5
    datetime(2023, 4, 1): 1.00,  # turn5
    datetime(2023, 5, 11): 1.00,
    datetime(2023, 6, 22): 1.00,
    datetime(2023, 6, 30): 1.00,  # turn6
    datetime(2023, 7, 1): 1.00,  # turn6
    datetime(2023, 8, 3): 1.00,
    datetime(2023, 9, 21): 1.00,
    datetime(2023, 9, 30): 1.00,  # turn7
    datetime(2023, 10, 1): 1.00,  # turn7
    datetime(2023, 11, 2): 1.00,
    datetime(2023, 12, 14): 1.00,
    datetime(2023, 12, 31): 1.00,  # turn8
    datetime(2024, 1, 1): 1.00,  # turn8
    datetime(2024, 2, 8): 1.00,  # estimated MPC dates..
    datetime(2024, 3, 21): 1.00,
    datetime(2024, 5, 16): 1.00,
    datetime(2024, 6, 20): 1.00,
    datetime(2024, 8, 8): 1.00,
    datetime(2024, 9, 19): 1.00,
    datetime(2024, 11, 7): 1.00,
    datetime(2024, 12, 12): 1.00
}
nodes_long = {
    datetime(2025, 3, 19): 1.00,  # long term tenors..
    datetime(2027, 3, 17): 1.00,
    datetime(2029, 3, 15): 1.00,
    datetime(2032, 3, 15): 1.00,
    datetime(2037, 3, 15): 1.00,
    datetime(2042, 3, 15): 1.00,
    datetime(2052, 3, 15): 1.00,
    datetime(2062, 3, 15): 1.00,
    datetime(2072, 3, 15): 1.00,
}
nodes = {**nodes_short, **nodes_long}
nodes_dual = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes.items())}

turns = {
    SwapSpread(
        Swap(datetime(2022, 3, 30), 1, 1, 1, days=True),
        Swap(datetime(2022, 3, 31), 1, 1, 1, days=True),
    ): -0.03,
    SwapSpread(
        Swap(datetime(2022, 3, 31), 1, 1, 1, days=True),
        Swap(datetime(2022, 4, 1), 1, 1, 1, days=True),
    ): 0.03,  # turn 1
    SwapSpread(
        Swap(datetime(2022, 6, 29), 1, 1, 1, days=True),
        Swap(datetime(2022, 6, 30), 1, 1, 1, days=True),
    ): -0.05,
    SwapSpread(
        Swap(datetime(2022, 6, 30), 1, 1, 1, days=True),
        Swap(datetime(2022, 7, 1), 1, 1, 1, days=True),
    ): 0.05,  # turn 2
    SwapSpread(
        Swap(datetime(2022, 9, 29), 1, 1, 1, days=True),
        Swap(datetime(2022, 9, 30), 1, 1, 1, days=True),
    ): -0.03,
    SwapSpread(
        Swap(datetime(2022, 9, 30), 1, 1, 1, days=True),
        Swap(datetime(2022, 10, 1), 1, 1, 1, days=True),
    ): 0.03,  # turn 3
    SwapSpread(
        Swap(datetime(2022, 12, 30), 1, 1, 1, days=True),
        Swap(datetime(2022, 12, 31), 1, 1, 1, days=True),
    ): -0.05,
    SwapSpread(
        Swap(datetime(2022, 12, 31), 1, 1, 1, days=True),
        Swap(datetime(2023, 1, 1), 1, 1, 1, days=True),
    ): 0.05,  # turn 4
    SwapSpread(
        Swap(datetime(2023, 3, 30), 1, 1, 1, days=True),
        Swap(datetime(2023, 3, 31), 1, 1, 1, days=True),
    ): -0.03,
    SwapSpread(
        Swap(datetime(2023, 3, 31), 1, 1, 1, days=True),
        Swap(datetime(2023, 4, 1), 1, 1, 1, days=True),
    ): 0.03,  # turn 5
    SwapSpread(
        Swap(datetime(2023, 6, 29), 1, 1, 1, days=True),
        Swap(datetime(2023, 6, 30), 1, 1, 1, days=True),
    ): -0.05,
    SwapSpread(
        Swap(datetime(2023, 6, 30), 1, 1, 1, days=True),
        Swap(datetime(2023, 7, 1), 1, 1, 1, days=True),
    ): 0.05,  # turn 6
    SwapSpread(
        Swap(datetime(2023, 9, 29), 1, 1, 1, days=True),
        Swap(datetime(2023, 9, 30), 1, 1, 1, days=True),
    ): -0.03,
    SwapSpread(
        Swap(datetime(2023, 9, 30), 1, 1, 1, days=True),
        Swap(datetime(2023, 10, 1), 1, 1, 1, days=True),
    ): 0.03,  # turn 7
    SwapSpread(
        Swap(datetime(2023, 12, 30), 1, 1, 1, days=True),
        Swap(datetime(2023, 12, 31), 1, 1, 1, days=True),
    ): -0.05,
    SwapSpread(
        Swap(datetime(2023, 12, 31), 1, 1, 1, days=True),
        Swap(datetime(2024, 1, 1), 1, 1, 1, days=True),
    ): 0.05,  # turn 8
}

swaps = {**ini_swaps, **turns, **curvature_conds}
weights = [1]*len(ini_swaps) + [1]*len(turns) + [0.0001]*len(curvature_conds)
s_cv_turns = SolvedCurve(
    nodes=nodes_dual, interpolation="log_linear",
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()),
    algorithm="levenberg_marquardt", 
    w=weights,
)
print(s_cv_turns.iterate())

In [ ]:
x = [datetime(2022,1,1) + i * timedelta(days=1) for i in range(365*5)]
y = [s_cv_turns.rate(date, days=1).real for date in x]

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlim(datetime(2022,9,1),datetime(2024,4,1))
ax.set_ylim(2.1, 2.8)
plt.xticks(rotation=90)
plt.show()

## Second layer

Adding in longer term granularity to the log-cubic spline section

In [ ]:
t_layer_1 = [
     datetime(2025, 3, 19), datetime(2025, 3, 19), datetime(2025, 3, 19), datetime(2025, 3, 19),
     datetime(2027, 3, 15),
     datetime(2029, 3, 15),
     datetime(2032, 3, 15),
     datetime(2037, 3, 15),
     datetime(2042, 3, 15),
     datetime(2052, 3, 15),
     datetime(2062, 3, 15),
     datetime(2072, 3, 15), datetime(2072, 3, 15), datetime(2072, 3, 15), datetime(2072, 3, 15),
    ]

adv_cv_1 = AdvancedCurve(
    nodes=nodes_dual, interpolation="log_linear", 
    swaps=list(swaps.keys()), obj_rates=list(swaps.values()), t=t_layer_1,
    algorithm="levenberg_marquardt", 
    w=weights,
)
print(adv_cv_1.iterate())

skews_layer_2 = {
    Swap(datetime(2022, 1, 1), 12*4, 12, 12): -0.0015,
    Swap(datetime(2022, 1, 1), 12*6, 12, 12): +0.0015,
    Swap(datetime(2022, 1, 1), 12*8, 12, 12): -0.0005,
    Swap(datetime(2022, 1, 1), 12*9, 12, 12): -0.0005,
    Swap(datetime(2022, 1, 1), 12*12, 12, 12): -0.001,
    Swap(datetime(2022, 1, 1), 12*25, 12, 12): 0,
    Swap(datetime(2022, 1, 1), 12*35, 12, 12): -0.0005,
    Swap(datetime(2022, 1, 1), 12*45, 12, 12): 0,
}

t_layer_2 = [
     datetime(2025, 3, 19), datetime(2025, 3, 19), datetime(2025, 3, 19), datetime(2025, 3, 19),
     datetime(2026, 3, 15),  # 4y
     datetime(2027, 3, 15),
     datetime(2028, 3, 15),  # 6y
     datetime(2029, 3, 15), 
     datetime(2030, 3, 15),  # 8y
     datetime(2031, 3, 15),  # 9y
     datetime(2032, 3, 15),
     datetime(2034, 3, 15),  # 12y
     datetime(2037, 3, 15),
     datetime(2042, 3, 15),
     datetime(2047, 3, 15),  # 25y
     datetime(2052, 3, 15),
     datetime(2057, 3, 15),  # 35y
     datetime(2062, 3, 15),
     datetime(2067, 3, 15),  # 45y
     datetime(2072, 3, 15), datetime(2072, 3, 15), datetime(2072, 3, 15), datetime(2072, 3, 15),
    ]

swaps_layer_2 = {
    swap: swap.rate(adv_cv_1).real + skew for (swap, skew) in skews_layer_2.items()
}

nodes_2 = {**nodes_short, **{
    datetime(2025, 3, 19): 1.00,  # long term tenors..
    datetime(2026, 3, 17): 1.00,
    datetime(2027, 3, 17): 1.00,
    datetime(2028, 3, 15): 1.00,
    datetime(2029, 3, 17): 1.00,
    datetime(2030, 3, 17): 1.00,
    datetime(2031, 3, 17): 1.00,
    datetime(2032, 3, 15): 1.00,
    datetime(2034, 3, 15): 1.00,
    datetime(2037, 3, 15): 1.00,
    datetime(2042, 3, 15): 1.00,
    datetime(2047, 3, 15): 1.00,
    datetime(2052, 3, 15): 1.00,
    datetime(2057, 3, 15): 1.00,
    datetime(2062, 3, 15): 1.00,
    datetime(2067, 3, 15): 1.00,
    datetime(2072, 3, 15): 1.00,
}}
nodes_dual_2 = {k: Dual(v, {f"v{i}": 1}) for i, (k,v) in enumerate(nodes_2.items())}

swaps_2 = {**ini_swaps, **swaps_layer_2, **turns, **curvature_conds}
weights_2 = [1]*(len(ini_swaps)+len(swaps_layer_2)+len(turns)) + [0.0001]*len(curvature_conds)
adv_cv_2 = AdvancedCurve(
    nodes=nodes_dual_2, interpolation="log_linear", 
    swaps=list(swaps_2.keys()), obj_rates=list(swaps_2.values()), t=t_layer_2,
    algorithm="levenberg_marquardt", 
    w=weights_2
)
print(adv_cv_2.iterate())

In [ ]:
x = [datetime(2024,6,15) + i * timedelta(days=1) for i in range(365*7)]
y = [adv_cv_1.rate(date, days=1).real for date in x]
z = [adv_cv_2.rate(date, days=1).real for date in x]
w = [s_cv.rate(date, days=1).real for date in x]

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, z)
ax.plot(x, w)
ax.set_xlim(datetime(2024,6,15),datetime(2031,9,30))
ax.set_ylim(2.05,2.30)
plt.xticks(rotation=90)
plt.show()

## Risk

In [ ]:
test_swap = Swap(datetime(2022,3,31), 1, 1, 1, days=True, notional=10e9) # Mar 22 Turn
# test_swap = Swap(datetime(2022, 9, 16), 3, 3, 3) # Sep IMM
# test_swap = Swap(datetime(2022, 5, 5), 42, 42, 42, days=True) # May MPC
risk = test_swap.risk(adv_cv_2)

In [ ]:
index = ["d1", "d2", "imm1", "imm2", "imm3", "imm4", "imm5", "imm6", "imm7",
         "imm8", "imm9", "imm10", "imm11", "imm12", "5y", "7y", "10y", "15y", "20y",
         "30y", "40y", "50y", "4y", "6y", "8y", "9y", "12y", "25y", "35y", "45y"]
for i in range(8):
    index += [f"turn{i+1}_left",  f"turn{i+1}_right"]
index += [f"curv{i+1}" for i in range(23)] 
pd.DataFrame({"swap": risk[:, 0]}, index=index).style